# **Content:**
* Recurrent Neural Network (RNN)<br>
1. [Implamenting RNN with Keras](#3)<br>
    * [Loading and Preprocessing Data](#4)<br>
    * [Create RNN Model](#5)<br>
    * [Predictions and Visualising RNN Model](#6)<br>
1. [Implementing Long Short Term Memory with Keras](#8)<br>
    * [Loading and Preprocessing Data](#9)<br>
    * [Preprocessing Data](#10)<br>
    * [Create LSTM Model](#11)<br>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="3"></a><br>
# **1.Implementing RNN with Keras**

<a id="4"></a><br>
## **-Loading and Preprocessing Data**

In [ ]:
data_train=pd.read_csv("/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv")

In [ ]:
data_train.head()

In [ ]:
train=data_train.loc[:,["Open"]].values
train

In [ ]:
#feature scaling 
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
train_scaled=scaler.fit_transform(train)
train_scaled

In [ ]:
plt.plot(train_scaled)
plt.show()

In [ ]:
#creating data structure
X_train=[]
y_train=[]
timesteps=50
for i in range(timesteps,1258):
    X_train.append(train_scaled[i-timesteps:i,0])
    y_train.append(train_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)

In [ ]:
#reshape
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train

In [ ]:
y_train

<a id="5"></a><br>
## **-Create RNN Model**

In [ ]:
# importing keras libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout

regressor=Sequential()

#adding first rnn layer
regressor.add(SimpleRNN(units=50,activation="tanh",return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))

#adding second rnn layer
regressor.add(SimpleRNN(units=50,activation='tanh',return_sequences=True))
regressor.add(Dropout(0.2))

#adding third rnn layer 
regressor.add(SimpleRNN(units=50,activation='tanh',return_sequences=True))
regressor.add(Dropout(0.2))

#adding fourth rnn layer
regressor.add(SimpleRNN(units=50))
regressor.add(Dropout(0.2))

#adding the output layer 
regressor.add(Dense(units=1))

#compiling the rnn
regressor.compile(optimizer="adam",loss="mean_squared_error")

#fitting the rnn to the training set
regressor.fit(X_train,y_train,epochs=100,batch_size=32)

<a id="6"></a><br>
## **-Predictions and Visualising RNN Model**

In [ ]:
data_test=pd.read_csv("/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv")
data_test.head()

In [ ]:
real_stock_price=data_test.loc[:,["Open"]].values
real_stock_price

In [ ]:
data_total=pd.concat((data_train['Open'],data_test['Open']),axis=0)
inputs=data_total[len(data_total)-len(data_test)-timesteps:].values.reshape(-1,1)
inputs=scaler.transform(inputs) # min max scaler
inputs

In [ ]:
X_test = []
for i in range(timesteps, 70):
    X_test.append(inputs[i-timesteps:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

#visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

<a id="8"></a><br>
# **2.Implementing LSTMs with Keras**

<a id="9"></a><br>
## **-Visualizing Data**

In [ ]:
import math 
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error

In [ ]:
data_train.head()

In [ ]:
train=data_train.iloc[:,1].values
plt.plot(train)
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.show()

<a id="11"></a><br>
## **-Create LSTMs Model**

In [ ]:
#model
model=Sequential()
model.add(LSTM(60,input_shape=(X_train.shape[1],1)))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer="adam")
model.fit(X_train,y_train,epochs=15,batch_size=1)

In [ ]:
trainPredict=model.predict(X_train)
testPredict=model.predict(X_test)

trainPredict=scaler.inverse_transform(trainPredict)
trainY=scaler.inverse_transform([y_train])
testPredict=scaler.inverse_transform(testPredict)

trainScore=math.sqrt(mean_squared_error(trainY[0],trainPredict[:,0]))
print("Train Score: %.2f RMSE" % (trainScore))

In [ ]:
#visualising 
plt.plot(real_stock_price,color="red",label="Real Google Stock Price")
plt.plot(testPredict,color="blue",label="Predict Google Stock Price")
plt.title("Google Stock Price Prediction")
plt.xlabel("Time"),
plt.ylabel("Google Stock Price")
plt.legend()
plt.show()